<a href="https://colab.research.google.com/github/giramakshay/retail_sales_prediction/blob/main/ML_Capstone_Regression_Retail_Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Name: Retail Sales Prediction

##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Contributor**     - Akshay Giram

# **Project Summary**



In this project we will develop a machine learning model to pridict sales for the Rossmann drug stores. Managers are required to have predictions of daily sales for upto six weeks in advance. This model will take into consideration various factors that affect sales, like promotions, competition, school and state holidays, seasonality, and locality.

The dataset contains 1 million entries for sales data across 1115 stores.

We will use linear regression to create a machine learning model for prediction of sales.